# Control My Robot |:-|

In [3]:
import numpy as np
import scipy as sp
from quaternion import from_rotation_matrix, quaternion

from scipy.spatial.transform import Rotation as R

from rlbench.environment import Environment
from rlbench.action_modes import ArmActionMode, ActionMode
from rlbench.observation_config import ObservationConfig, CameraConfig
from rlbench.tasks import PutGroceriesInCupboard
from pyrep.const import ConfigurationPathAlgorithms as Algos
import pprint
import time

import copy

from utils import RandomAgent, NoisyObjectPoseSensor, VisionObjectPoseSensor, RobotController

# Start the simulation, Initialize the environment

In [ ]:
### Set Action Mode
action_mode = ActionMode(ArmActionMode.ABS_JOINT_POSITION) # See rlbench/action_modes.py for other action modes

### Initialize Environment with Action Mode and desired observations
env = Environment(action_mode, '', ObservationConfig(), False)

### Load task into the environment
task = env.get_task(PutGroceriesInCupboard)

### Create Agent: TODO
agent = RandomAgent()

### Object Pose Sensor
obj_pose_sensor = NoisyObjectPoseSensor(env)

### Robot Controller Object
robot_controller = RobotController(env, task)

# Keep frequently used variables here

In [ ]:
### Useful variables
gripper_vertical_orientation = np.array([3.13767052e+00, 1.88300957e-03, 9.35417891e-01])
reset_position = env._robot.arm.get_tip().get_position()
reset_orientation = env._robot.arm.get_tip().get_orientation()

mustard_orientation = [ 7.07037210e-01,  7.07173109e-01, -6.37740828e-04, -2.06269184e-03]
mustard_position = [ 0.31035879, -0.12106754,  0.90185165]



In [ ]:
descriptions, obs = task.reset()


In [ ]:
crackers_position, crackers_orientation, crackers_obj = robot_controller.get_pose_and_object_from_simulation("spam_grasp_point")
crackers_rot = R.from_euler('xyz', crackers_orientation)

In [ ]:
## Motion 1
## (1) Sense mustard_grasp_point location. (2) Move gripper to a point 0.1m over mustard_grasp_point, while making it remain vertical.

#(1)
next_position, next_orientation, spam_obj = robot_controller.get_pose_and_object_from_simulation("spam_grasp_point")

#(2)
next_position[2] += 0.1
next_orientation = gripper_vertical_orientation
motion_1_plan = robot_controller.move(next_position, next_orientation)

In [ ]:
_ = robot_controller.translate(z=-0.1, ignore_collisions=False)

In [ ]:
_ = robot_controller.actuate_gripper(0)

In [ ]:
_ = robot_controller.translate(z=0.02, ignore_collisions=True)

In [ ]:
_ =robot_controller.translate(x=0.1, y=0.03, ignore_collisions=False)

In [ ]:
_ =robot_controller.translate(y=-0.1, ignore_collisions=False)

In [ ]:
_ =robot_controller.rotate_to([gripper_vertical_orientation[0], gripper_vertical_orientation[1], 2.87], gripper_state=1, ignore_collisions=False)

# Rotation Tools


In [ ]:
next_position, next_orientation, spam_obj = robot_controller.get_pose_and_object_from_simulation("spam_grasp_point")
# next_position[2] += 0.1
# next_orientation = gripper_vertical_orientation
# motion_1_plan = robot_controller.move(next_position, next_orientation)

In [ ]:
spam_obj = [obj for obj in task._task.get_graspable_objects() if obj.get_name() == 'spam'][0]
mustard_obj = [obj for obj in task._task.get_graspable_objects() if obj.get_name() == 'mustard'][0]

In [ ]:
## Motion 1
## (1) Sense mustard_grasp_point location. (2) Move gripper to a point 0.1m over mustard_grasp_point, while making it remain vertical.

#(1)
next_position, next_orientation, spam_obj = robot_controller.get_pose_and_object_from_simulation("waypoint3")

#(2)
# next_position[2] += 0.1
# next_orientation = gripper_vertical_orientation
motion_1_plan = robot_controller.move(next_position, next_orientation)

In [ ]:
spam_rot = R.from_euler('xyz', spam_obj.get_orientation())
desired_spam_rot = R.from_euler('xyz', [0,0,0])#crackers_rot
req_rotation = desired_spam_rot.inv() * spam_rot
# print(spam_rot.as_euler('xyz'), req_rotation.as_euler('xyz'))

end_effector_orintation = R.from_euler('xyz', env._robot.arm.get_tip().get_orientation())
new_orientation =  req_rotation * end_effector_orintation
new_orientation = list(new_orientation.as_euler('xyz'))
# print(end_effector_orintation.as_euler('xyz'), new_orientation.as_euler('xyz'))

robot_controller.rotate_to(new_orientation, ignore_collisions=False)


In [ ]:
robot_controller.rotate_to([0,0,0], ignore_collisions=False)


In [ ]:
robot_controller.move(reset_position, spam_obj.get_orientation())

In [ ]:
env._robot.arm._ik_tip = spam_obj

In [ ]:
spam_obj.get_orientation()

In [ ]:
env._robot.arm.get_tip().get_orientation(), env._robot.arm.get_tip().get_position()

In [ ]:
env._robot.arm._ik_tip = spam_obj

In [ ]:
next_position, next_orientation, _ = robot_controller.get_pose_and_object_from_simulation("waypoint3")

#(2)
# next_position[2] += 0.1
# next_orientation = gripper_vertical_orientation
motion_1_plan = robot_controller.move(next_position, next_orientation)

In [ ]:
spam_obj = [obj for obj in task._task.get_graspable_objects() if obj.get_name() == 'spam'][0]
env._robot.arm._ik_target = spam_obj
next_position, next_orientation, _ = robot_controller.get_pose_and_object_from_simulation("waypoint3")

#(2)
# next_position[2] += 0.1
# next_orientation = gripper_vertical_orientation
motion_1_plan = robot_controller.move(next_position, next_orientation)